In [1]:
# install necessory packages
!pip install pandas
!pip install numpy
!pip install chart-studio
!pip install cufflinks
!pip install seaborn
!pip install matplotlib
!pip install sklearn

# for save this in html format
!pip install nbconvert

In [19]:
# import neccesory packages

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import colorlover as cl # generating new colors
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
from plotly.subplots import make_subplots

import os

%matplotlib inline

# Plotly work in Jupyter Notebook

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


# Use Plotly locally

cf.go_offline()

In [3]:
# data reading Via button Imports

import ipywidgets as widgets
from IPython.display import display
import io

In [4]:
# Upload data file 
uploader = widgets.FileUpload(accept='.csv')
uploaded_dataframe = None

def handle_upload(change):
    global uploaded_dataframe
    try:   
        uploaded_file = uploader.value
        content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        data = pd.read_csv(io.BytesIO(content))
        uploaded_dataframe = data
        print('Msg :: Upload Successful')
    except:
        uploaded_dataframe = None
        print( 'Err :: Uploading Failed' )
    # Process the uploaded file
    # Perform any required data manipulation or analysis
    
    
uploader.observe(handle_upload, names='value')


display(uploader)

FileUpload(value={}, accept='.csv', description='Upload')

Msg :: Upload Successful


In [5]:
# Get The Data of the Uploaded Dataframe and store it into df Variable
df = uploaded_dataframe

In [6]:
df.head()

,Device_id,time,RPM,Total_rotations,On_time,Off_time
0,1,11-05-2023 00:01,10,10,1,0
1,1,11-05-2023 00:02,15,25,2,0
2,1,11-05-2023 00:03,15,40,3,0
3,1,11-05-2023 00:04,15,55,4,0
4,1,11-05-2023 00:05,15,70,5,0


In [49]:
# APIs

class DataVisual:
    
    def __init__(self, dataFrame):
        self.df = dataFrame
        isReady = self.validate()
        self.saveLocal = False
        self.directoryPath = None
        if(not isReady):
            print('War :: Invalid Data')
            return 
        print('Msg :: Data Validated')
        self.preCompute()
            
        
    def validate(self):
        if(self.df is None):
            print( 'Err :: DataFrame is None' )
            return False
        cols = list(self.df.columns)
        match = ['Device_id', 'time', 'RPM', 'Total_rotations', 'On_time', 'Off_time']
        try:
            result = all(x == y for x, y in zip(cols, match))
            if(result):
                return True
        except:
            print('Error : Data Mutation')
            
        return False
    
    
    def preCompute(self):
        # Change Columns Name
        changeName = {
            'Device_id' : 'id',
            'time' : 'time',
            'RPM' :'rpm',
            'Total_rotations':'total_rotations',
            'On_time' : 'on_time',
            'Off_time':'off_time'
        }
        self.df = self.df.rename(columns = changeName)
        
        # Adjest datatime object
        
        self.df['time'] = pd.to_datetime(self.df['time'])
        self.df['time'] = self.df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Calculate total_active_time = on_time - off_time 
        
        self.df['total_active_time'] = self.df['on_time'] - self.df['off_time']
        
        
        # Calculate total_rotation_time = total_rotation / RMP
        
        self.df['total_rotation_time'] = self.df['total_rotations'] / self.df['rpm']
        
        
        # Create Duty_Cycle 
        
        self.df['duty_cycle'] = (self.df['total_rotation_time']  / self.df['total_active_time'] )* 100
        
        
        # Create Group Based on IDs
        
        self.groupby_ID_df = self.df.groupby('id')
        
        
        # list of IDs
        
        self.unique_ids = self.df['id'].unique()
        
        
        # print 
        display(self.df)
        
    def save(self,value,path):
        self.saveLocal = True
        self.directoryPath = path
        

# Complete The agreegated display first 
    def agreegated_RPM(self):
        
        X = []
        Y = []
        Found = False
        
        n = len(self.unique_ids)
        colors = cl.scales[str(n)]['qual']['Set3']
        
        for group_name in self.unique_ids:
            try:
                Found = True
                group_data_all = self.groupby_ID_df.get_group(group_name)
                group_data = group_data_all[ group_data_all['rpm'] > 0 ]
                
                objectName = 'Device_ID_{}_'.format(group_name)
                
                Y.append( group_data['rpm'].round(4).max() )
                X.append( objectName + "Maximum" )
                
                Y.append( group_data['rpm'].round(4).mean() )
                X.append( objectName + "Average" )
                
                Y.append( group_data['rpm'].round(4).min() )
                X.append( objectName + "Minimum" )
                
            except KeyError:
                Found = False
                print("Err :: Group {} does not exist".format(group_name))
                break
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return

        
        fig = go.Figure(data=go.Bar(x=X, y=Y))
        fig.update_layout(title='Agreegated RPM of Devices', xaxis_title='Measure', yaxis_title='Value', template="plotly_dark")
        
        # save the. HTML FILE
#         offline.plot(fig, filename='agreegated_RPM.html', auto_open=False)

        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/agreegated_RPM.html", "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/agreegated_RPM.html", auto_open=False)
        
        return fig
            
    def agreegated_duty_cycle(self):
        Plots = []
        Found = False
        
        n = len(self.unique_ids)
        colors = cl.scales[str(n)]['qual']['Set3']
        cl_id = 0
        
        for group_name in self.unique_ids:
            try:
                Found = True
                group_data_all = self.groupby_ID_df.get_group(group_name)
                
                condition1 = group_data_all['rpm'] > 0 
                condition2 = group_data_all['duty_cycle'] <= 100
                
#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
                group_data = group_data_all[ condition1 & condition2 ]
                
                objectName = 'Device_ID_{}_'.format(group_name)
                
                
                Y = group_data['duty_cycle']
                X = group_data['time'] 
                Name = objectName + "Duty_Cycle"
                
                scatter = go.Scatter(
                    x=X,
                    y=Y,
                    mode='lines+markers',
                    line=dict(shape='spline', smoothing=1.3),
                    marker=dict(color=colors[cl_id]),
                    name=Name
                )
                
                Plots.append(scatter)
                
                cl_id += 1
                
                
            except KeyError:
                Found = False
                print("Err :: Group {} does not exist".format(group_name))
                break
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        layout = go.Layout(
            title='Agreegated Duty_Cycle',
            xaxis=dict(title='time'),
            yaxis=dict(title='duty_cycle_percentage', range=[75,110]),
            hovermode='closest',
            template="plotly_dark",
        )
        
        fig = go.Figure(data=Plots, layout=layout)
#         offline.plot(fig, filename='agreegated_duty_cycle.html', auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/agreegated_duty_cycle.html", "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/agreegated_duty_cycle.html", auto_open=False)
        
        return fig
        
        
# create individual 

    def RPM_vs_TIME(self, group_name):
        X = []
        Y = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

            condition1 = group_data_all['rpm'] > 0 
            condition2 = group_data_all['duty_cycle'] <= 100

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all[ condition1 & condition2 ]

            objectName = 'Device_ID_{}_'.format(group_name)


            Y = group_data['rpm']
            X = group_data['time'] 
            Name = objectName + "RPM_vs_TIME"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        scatter = go.Scatter(
            x=X,
            y=Y,
            mode='lines+markers',
            line=dict(shape='spline', smoothing=1.3),
            name=Name
        )

        layout = go.Layout(
            title='RPM_vs_TIME for device {}'.format(group_name),
            xaxis=dict(title='time'),
            yaxis=dict(title='rpm'),
            hovermode='closest',
            template="plotly_dark",
        )
        
        fig = go.Figure(data=[scatter], layout=layout)
#         offline.plot(fig, filename='RPM_vs_TIME_{}.html'.format(group_name), auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/RPM_vs_TIME_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/RPM_vs_TIME_{}.html".format(group_name), auto_open=False)
        
        return fig

    def DutyCycle_vs_Time(self, group_name):
        X = []
        Y = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

            condition1 = group_data_all['rpm'] > 0 
            condition2 = group_data_all['duty_cycle'] <= 100

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all[ condition1 & condition2 ]

            objectName = 'Device_ID_{}_'.format(group_name)


            Y = group_data['duty_cycle']
            X = group_data['time'] 
            Name = objectName + "DutyCycle_vs_Time"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        scatter = go.Scatter(
            x=X,
            y=Y,
            mode='lines+markers',
            line=dict(shape='spline', smoothing=1.3),
            name=Name
        )

        layout = go.Layout(
            title='DutyCycle_vs_Time for device {}'.format(group_name),
            xaxis=dict(title='time'),
            yaxis=dict(title='Duty_Cycle'),
            hovermode='closest',
            template="plotly_dark",
        )
        
        fig = go.Figure(data=[scatter], layout=layout)
#         offline.plot(fig, filename='DutyCycle_vs_Time_{}.html'.format(group_name), auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/DutyCycle_vs_Time_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/DutyCycle_vs_Time_{}.html".format(group_name), auto_open=False)
                
        return fig
    
    def TotalRotation_vs_RMP(self, group_name):
        X = []
        Y = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

            condition1 = group_data_all['rpm'] > 0 
            condition2 = group_data_all['duty_cycle'] <= 100

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all[ condition1 & condition2 ]

            objectName = 'Device_ID_{}_'.format(group_name)


            Y = group_data['rpm']
            X = group_data['total_rotations'] 
            Name = objectName + "TotalRotation_vs_RMP"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        scatter = go.Scatter(
            x=X,
            y=Y,
            mode='lines+markers',
            line=dict(shape='spline', smoothing=1.3),
            name=Name
        )

        layout = go.Layout(
            title='TotalRotation_vs_RMP for device {}'.format(group_name),
            xaxis=dict(title='TotalRotation'),
            yaxis=dict(title='RPM'),
            hovermode='closest',
            template="plotly_dark"
        )
        
        fig = go.Figure(data=[scatter], layout=layout)
#         offline.plot(fig, filename='TotalRotation_vs_RMP_{}.html'.format(group_name), auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/TotalRotation_vs_RMP_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/TotalRotation_vs_RMP_{}.html".format(group_name), auto_open=False)
                
        return fig
    
    
    def OnTime_and_OffTime(self, group_name):
        X = []
        Y = []
        x = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all

            objectName = 'Device_ID_{}_'.format(group_name)


            Y = group_data['on_time']
            X = group_data['off_time'] 
            x = group_data['time']
            Name = objectName + "OnTime_and_OffTime"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        x = np.arange(len(Y))
        y = np.zeros_like(x)
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=x,
            y=Y,
            mode='lines',
            stackgroup='one',
            name='On Time'
        ))
        
        fig.add_trace(go.Scatter(
            x=x,
            y=X,
            mode='lines',
            stackgroup='one',
            name='Off Time'
        ))

        fig.update_layout(
            title='Stacked Area Chart For OnTime and OffTime on {}'.format(group_name),
            xaxis=dict(title='Time'),
            yaxis=dict(title='Duration'),
            hovermode='x',
            template="plotly_dark",
        )

#         offline.plot(fig, filename='OnTime_and_OffTime_{}.html'.format(group_name), auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/OnTime_and_OffTime_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/OnTime_and_OffTime_{}.html".format(group_name), auto_open=False)
                
        return fig
    
    
    def RPM_distribution(self, group_name):
        data = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all

            objectName = 'Device_ID_{}_'.format(group_name)


            data = group_data['rpm']
            Name = objectName + "RPM_distribution"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        
        histogram = go.Histogram(
            x=data,
            nbinsx=len(set(data)), 
            histnorm='probability'
        )

        
        fig = go.Figure(data=[histogram])
        fig.update_layout(
            title='RPM_distribution Chart for {}'.format(group_name),
            xaxis=dict(title='Value'),
            yaxis=dict(title='Probability'),
            template="plotly_dark",
        )
        
#         offline.plot(fig, filename='RPM_distribution_{}.html'.format(group_name), auto_open=False)
        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/RPM_distribution_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/RPM_distribution_{}.html".format(group_name), auto_open=False)
                
        return fig
    
    def K_mean(self, group_name, K):
        X = []
        Name = None
        Found = False
        
        try:
            Found = True
            group_data_all = self.groupby_ID_df.get_group(group_name)

            condition1 = group_data_all['rpm'] > 0 
            condition2 = group_data_all['duty_cycle'] <= 100

#                 group_data = group_data_all [ [group_data_all[ group_data_all['rpm'] > 0 ].bool() and [group_data_all['duty_cycle'] <= 100].bool() ]] 
            group_data = group_data_all[ condition1 & condition2 ]

            objectName = 'Device_ID_{}_'.format(group_name)


            X = group_data[['rpm','total_rotations','total_active_time']]
            Name = objectName + "K_mean"


        except KeyError:
            Found = False
            print("Err :: Group {} does not exist".format(group_name))
            
        
        
        if( not Found ):
            print('Err :: Nothing to Display')
            return
        
        kmeans = KMeans(n_clusters=K)
        kmeans.fit(X)
        
        labels = kmeans.labels_
        
        group_data['cluster'] = labels
        
        scatter = go.Scatter3d(
            x=group_data['rpm'],
            y=group_data['total_rotations'],
            z=group_data['total_active_time'],
            mode='markers',
            marker=dict(
                color=group_data['cluster'],
                size=5,
                colorscale='Viridis',
                opacity=0.8
            ),
            name='Data Points'
        )
        
        layout = go.Layout(
            title='K-means Clustering For device {}'.format(group_name),
            scene=dict(
                xaxis=dict(title='RPM'),
                yaxis=dict(title='Total Rotations'),
                zaxis=dict(title='Total Active Time')
            ),
            template="plotly_dark",
        )
        
        fig = go.Figure(data=[scatter], layout=layout)
        
#         offline.plot(fig, filename='K_mean_{}.html'.format(group_name), auto_open=False)

        if(self.saveLocal):
            if not os.path.exists(self.directoryPath):
                os.mkdir(self.directoryPath)
            with open(self.directoryPath+"/K_mean_{}.html".format(group_name), "w") as f:
                offline.plot(fig, filename=self.directoryPath+"/K_mean_{}.html".format(group_name), auto_open=False)
        return fig
        
    def ALL(self):
        f = []
        f1 = self.agreegated_RPM()
        f2 = self.agreegated_duty_cycle()
        
        f.append(f1)
        f.append(f2)

        K = 3
        for ids in self.unique_ids:
            f.append(self.RPM_vs_TIME(ids))
            f.append(self.DutyCycle_vs_Time(ids))
            f.append(self.TotalRotation_vs_RMP(ids))
            f.append(self.OnTime_and_OffTime(ids))
            f.append(self.RPM_distribution(ids))
#             f.append(self.K_mean(ids, K))

        # Create a new subplot figure with vertically stacked subplots and spacing
        subplot_fig = make_subplots(rows=len(f), cols=1, vertical_spacing=0.04)

        # Add each figure as a subplot
        for i, fig in enumerate(f, start=1):
            for trace in fig.data:
                subplot_fig.add_trace(trace, row=i, col=1)

        # Update the subplot figure's layout
        subplot_fig.update_layout(height=800, width=800, title="Merged Figures")

        # Get the HTML representation of the merged figures
        html_content = subplot_fig.to_html(full_html=False)

        # Display the HTML content
#         display(html_content)

        with open('merged_figures.html', 'w') as file:
            file.write(html_content)
            
    def monitor(self):
        f = []
        f1 = self.agreegated_RPM()
        f2 = self.agreegated_duty_cycle()
        
        f.append(f1)
        f.append(f2)

        K = 3
        for ids in self.unique_ids:
            f.append(self.RPM_vs_TIME(ids))
            f.append(self.DutyCycle_vs_Time(ids))
            f.append(self.TotalRotation_vs_RMP(ids))
            f.append(self.OnTime_and_OffTime(ids))
            f.append(self.RPM_distribution(ids))
            f.append(self.K_mean(ids, K))
            
        # we have a list of figure
        # which are been stored at the local storage
        
        # do visualize all togetgher 
        
        for figure in f:
            display(figure)

### Test APIs

In [50]:
vis = DataVisual(df)
vis.save(True,"images")

Msg :: Data Validated


,id,time,rpm,total_rotations,on_time,off_time,total_active_time,total_rotation_time,duty_cycle
0,1,2023-11-05 00:01:00,10,10,1,0,1,1.000000,100.000000
1,1,2023-11-05 00:02:00,15,25,2,0,2,1.666667,83.333333
2,1,2023-11-05 00:03:00,15,40,3,0,3,2.666667,88.888889
3,1,2023-11-05 00:04:00,15,55,4,0,4,3.666667,91.666667
4,1,2023-11-05 00:05:00,15,70,5,0,5,4.666667,93.333333
...,...,...,...,...,...,...,...,...,...
463,3,2023-11-05 02:31:00,15,1554,121,31,90,103.600000,115.111111
464,3,2023-11-05 02:32:00,15,1569,122,31,91,104.600000,114.945055
465,3,2023-11-05 02:33:00,15,1584,123,31,92,105.600000,114.782609
466,3,2023-11-05 02:34:00,15,1599,124,31,93,106.600000,114.623656


In [51]:
vis.monitor()

In [12]:
# Display
# vis.agreegated_RPM().to_html()

In [42]:
# vis.agreegated_RPM()

In [43]:
# vis.agreegated_duty_cycle()

In [13]:
ids = 1 # you can change the id here

In [44]:
# vis.RPM_vs_TIME(ids)

In [45]:
# vis.DutyCycle_vs_Time(ids)

In [46]:
# vis.OnTime_and_OffTime(ids)

In [47]:
# vis.RPM_distribution(ids)

In [48]:
# vis.K_mean(ids,3)